In [1]:
import random
from dataclasses import replace

from pathlib import Path
import sys

cwd = Path.cwd()
for root in [cwd, *cwd.parents]:
    src_path = root / 'src'
    if (src_path / 'replenishment').exists():
        sys.path.insert(0, str(src_path))
        break
    src_path = root / 'replenishment' / 'src'
    if (src_path / 'replenishment').exists():
        sys.path.insert(0, str(src_path))
        break

for name in list(sys.modules):
    if name == 'replenishment' or name.startswith('replenishment.'):
        del sys.modules[name]

In [2]:
import time
from collections import Counter

from replenishment import ForecastCandidatesConfig, optimize_forecast_targets, optimize_aggregation_and_forecast_targets

In [3]:
article_count = 50_000
periods = 120
lead_time = 1
holding_cost_per_unit = 0.8
stockout_cost_per_unit = 3.5

base_demand = [50 + (idx % 10) for idx in range(periods)]
percentile_targets = list(range(10, 85, 5))

def _percentile_offset(target: int) -> int:
    return int(round((target - 50) / 5))

forecast_candidates = {
    f"p{target}": [value + _percentile_offset(target) for value in base_demand]
    for target in percentile_targets
}

candidate_configs = {
    f'article-{idx:05d}': ForecastCandidatesConfig(
        periods=periods,
        demand=base_demand,
        initial_on_hand=40,
        lead_time=lead_time,
        forecast_candidates=forecast_candidates,
        holding_cost_per_unit=holding_cost_per_unit,
        stockout_cost_per_unit=stockout_cost_per_unit,
    )
    for idx in range(article_count)
}


In [4]:
start = time.perf_counter()
candidate_windows = list(range(1, 11))
optimized_targets = optimize_aggregation_and_forecast_targets(
    candidate_configs,
    candidate_windows=candidate_windows,
)
elapsed = time.perf_counter() - start

print(f'Total time: {elapsed:.2f}s')
print(f'Time per article: {elapsed / article_count:.6f}s')

target_counts = Counter(result.target for result in optimized_targets.values())
print(dict(target_counts))


Total time: 603.19s
Time per article: 0.012064s
{'p55': 50000}


In [5]:
optimized_targets['article-00000']

AggregationForecastTargetOptimizationResult(window=2, target='p55', policy=PercentileForecastOptimizationPolicy(forecast=[103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119, 103, 107, 111, 115, 119], lead_time=1), simulation=SimulationResult(snapshots=[InventorySnapshot(period=0, starting_on_hand=40, demand=101, received=0, ending_on_hand=0, backorders=61, order_placed=107, on_order=107), InventorySnapshot(period=1, starting_on_hand=46, demand=105, received=107, ending_on_hand=0, backorders=59, order_placed=111, on_order=111), InventorySnapshot(period=2, starting_on_hand=52, demand=109, received=111, ending_on_hand=0, backorders=57, order_placed=115, on_order=115), InventorySnapshot(period=3, starting_on_hand=58, demand=113, received=115, ending_on_hand=0, backorders

In [6]:
{article_id: result.target for article_id, result in optimized_targets.items()}


{'article-00000': 'p55',
 'article-00001': 'p55',
 'article-00002': 'p55',
 'article-00003': 'p55',
 'article-00004': 'p55',
 'article-00005': 'p55',
 'article-00006': 'p55',
 'article-00007': 'p55',
 'article-00008': 'p55',
 'article-00009': 'p55',
 'article-00010': 'p55',
 'article-00011': 'p55',
 'article-00012': 'p55',
 'article-00013': 'p55',
 'article-00014': 'p55',
 'article-00015': 'p55',
 'article-00016': 'p55',
 'article-00017': 'p55',
 'article-00018': 'p55',
 'article-00019': 'p55',
 'article-00020': 'p55',
 'article-00021': 'p55',
 'article-00022': 'p55',
 'article-00023': 'p55',
 'article-00024': 'p55',
 'article-00025': 'p55',
 'article-00026': 'p55',
 'article-00027': 'p55',
 'article-00028': 'p55',
 'article-00029': 'p55',
 'article-00030': 'p55',
 'article-00031': 'p55',
 'article-00032': 'p55',
 'article-00033': 'p55',
 'article-00034': 'p55',
 'article-00035': 'p55',
 'article-00036': 'p55',
 'article-00037': 'p55',
 'article-00038': 'p55',
 'article-00039': 'p55',
